In [ ]:
# Install necessary libraries:
!pip install pandas geopandas matplotlib folium
!pip install geopy

In [ ]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Point
import folium
from folium.plugins import MarkerCluster
from geopy.geocoders import Nominatim
import logging
import time


# Load your dataset
df = pd.read_csv('weather_data.csv')
df = df.head(500)

In [ ]:

# Extract unique city names from the 'Location' column
unique_cities = df['Location'].unique()

# Initialize the geocoder
geolocator = Nominatim(user_agent="geoapiExercises")

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger()

# Function to get coordinates with error handling
def get_coordinates(city, retries=3):
    for attempt in range(retries):
        try:
            location = geolocator.geocode(city)
            if location:
                return (location.latitude, location.longitude)
            else:
                logger.warning(f"Could not find coordinates for {city}")
                return (None, None)
        except Exception as e:
            logger.error(f"Error fetching coordinates for {city}: {e}")
            time.sleep(1)  # Wait a bit before retrying
    return (None, None)

# Create a dictionary of city coordinates
city_coords = {}
for city in unique_cities:
    city_coords[city] = get_coordinates(city)
    time.sleep(1)  # To avoid overwhelming the server with requests

# Convert to a DataFrame
coords_df = pd.DataFrame(city_coords).T.reset_index()
coords_df.columns = ['Location', 'Latitude', 'Longitude']

print(coords_df)


In [ ]:

# Create a DataFrame with the coordinates
coords = pd.DataFrame(city_coords).T.reset_index()
coords.columns = ['Location', 'Latitude', 'Longitude']

# Merge coordinates with the main DataFrame
df = df.merge(coords, on='Location')

# Create a GeoDataFrame
geometry = [Point(xy) for xy in zip(df['Longitude'], df['Latitude'])]
gdf = gpd.GeoDataFrame(df, geometry=geometry)

# Set the coordinate reference system to WGS84
gdf.crs = {'init': 'epsg:4326'}


In [ ]:
# Plot the locations
gdf.plot(column='Temperature_C', cmap='coolwarm', legend=True, markersize=5)
plt.title('Temperature Distribution by Location')
plt.show()


In [ ]:
import folium
from folium.plugins import MarkerCluster

# Initialize a Folium map
m = folium.Map(location=[20,0], zoom_start=2)


In [ ]:
# Add points to the map
marker_cluster = MarkerCluster().add_to(m)


In [ ]:
# Create and add markers
for idx, row in df.iterrows():
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=folium.Popup(
            f"Location: {row['Location']}<br>"
            f"Temperature: {row['Temperature_C']} C<br>"
            f"Humidity: {row['Humidity_pct']}%<br>"
            f"Precipitation: {row['Precipitation_mm']} mm<br>"
            f"Wind Speed: {row['Wind_Speed_kmh']} km/h",
            max_width=300
        )
    ).add_to(marker_cluster)

In [ ]:
# Save to an HTML file
m.save('weather_map.html')

# Display the map in a Jupyter notebook
m